In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2 as cv
import os
from  tqdm import tqdm
from keras.applications import InceptionV3,MobileNetV2

In [2]:
base_dir = r'C:\Users\Parth Tripathi\Desktop\DS\Sign_language_detector'

In [3]:
train_dir = fr'{base_dir}\asl_dataset'

In [4]:
zero = fr'{train_dir}\0'
one = fr"{train_dir}\1"
two = fr'{train_dir}\2'
three = fr'{train_dir}\3'
four = fr'{train_dir}\4'
five = fr'{train_dir}\5'
six = fr'{train_dir}\6'
seven = fr'{train_dir}\7'
eight = fr'{train_dir}\8'
nine = fr'{train_dir}\9'

a = fr'{train_dir}\a'
b = fr'{train_dir}\b'
c = fr'{train_dir}\c'
d = fr'{train_dir}\d'
e = fr'{train_dir}\e'
f = fr'{train_dir}\f'
g = fr'{train_dir}\g'
h = fr'{train_dir}\h'
i = fr'{train_dir}\i'
j = fr'{train_dir}\j'
k = fr'{train_dir}\k'
l = fr'{train_dir}\l'
m = fr'{train_dir}\m'
n = fr'{train_dir}\n'
o = fr'{train_dir}\o'
p = fr'{train_dir}\p'
q = fr'{train_dir}\q'
r = fr'{train_dir}\r'
s = fr'{train_dir}\s'
t = fr'{train_dir}\t'
u = fr'{train_dir}\u'
v = fr'{train_dir}\v'
w = fr'{train_dir}\w'
x = fr'{train_dir}\x'
y = fr'{train_dir}\y'
z = fr'{train_dir}\z'


In [5]:
labels = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']


train = []
y_train=[]

img_size = 224
for i in labels:
    folderpath = f"{train_dir}\{i}"
    for j in tqdm(os.listdir(folderpath)):
        img = cv.imread(os.path.join(folderpath,j))
        img = cv.resize(img,(img_size,img_size))
        train.append(img)
        y_train.append(i)


train = np.array(train)
y_train = np.array(y_train)        

100%|██████████| 70/70 [00:00<00:00, 115.29it/s]


In [6]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test = train_test_split(train,y_train, test_size=0.1,random_state=101)


y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)



print(X_test.shape)
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)


(252, 224, 224, 3)
(2263, 224, 224, 3)
(2263, 36)
(252, 36)


In [7]:
'''
pre_model = InceptionV3(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)
'''


pre_model = MobileNetV2(
        input_shape=(224,224,3),
        weights='imagenet',
        include_top=False
)       



for layers in pre_model.layers:
    layers.trainable = False


last_output = pre_model.layers[-2].output  

In [19]:
from keras.optimizers import RMSprop,Adam
from keras import Model

x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
output = tf.keras.layers.Dense(len(labels),activation='softmax')(x)

model = Model(pre_model.input,output)

model.compile(optimizer=Adam(learning_rate=0.001), loss = 'categorical_crossentropy',metrics = ['accuracy'])


In [20]:
class callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if(logs.get('val_accuracy')>0.99):
            self.model.stop_training=True

In [21]:
from keras.preprocessing.image import ImageDataGenerator

callback = callbacks()

datagen = ImageDataGenerator(
        rescale=(1./255),
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

datagen_val = ImageDataGenerator(rescale=1./255)

model.fit(
        datagen.flow(X_train,y_train),
        epochs=10,
        validation_data=datagen_val.flow(X_train,y_train),
        callbacks = [callback]
)        


Epoch 1/10
71/71 [==============================] - 110s 1s/step - loss: 1.9441 - accuracy: 0.5365 - val_loss: 3.0810 - val_accuracy: 0.2338
Epoch 2/10
71/71 [==============================] - 97s 1s/step - loss: 0.7067 - accuracy: 0.8648 - val_loss: 1.1234 - val_accuracy: 0.7251
Epoch 3/10
71/71 [==============================] - 93s 1s/step - loss: 0.3770 - accuracy: 0.9333 - val_loss: 0.4050 - val_accuracy: 0.9381
Epoch 4/10
71/71 [==============================] - 94s 1s/step - loss: 0.2518 - accuracy: 0.9496 - val_loss: 0.2568 - val_accuracy: 0.9417
Epoch 5/10
71/71 [==============================] - 93s 1s/step - loss: 0.1793 - accuracy: 0.9646 - val_loss: 0.1803 - val_accuracy: 0.9585
Epoch 6/10
71/71 [==============================] - 92s 1s/step - loss: 0.1323 - accuracy: 0.9695 - val_loss: 0.1306 - val_accuracy: 0.9717
Epoch 7/10
71/71 [==============================] - 92s 1s/step - loss: 0.1286 - accuracy: 0.9691 - val_loss: 0.2512 - val_accuracy: 0.9289
Epoch 8/10
12/71 [=

KeyboardInterrupt: 

In [15]:
#model.save('sign_detector_mobilenet.h5')

In [16]:
'''
tf_model_file = "sign_detector_mobilenetV3.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

with open(tf_model_file,'wb') as file:
    file.write(tflite_model)
'''  

INFO:tensorflow:Assets written to: C:\Users\91976\AppData\Local\Temp\tmpwunr7jh1\assets


INFO:tensorflow:Assets written to: C:\Users\91976\AppData\Local\Temp\tmpwunr7jh1\assets


predicting

In [17]:
t = []

img = cv.imread(r"C:\Users\Parth Tripathi\Desktop\DS\Sign_language_detector\asl_dataset\3\hand1_3_bot_seg_3_cropped.jpeg")

img = cv.resize(img,(224,224))
t.append(img)

t = np.array(t)

In [18]:
t = t / 255.0
pred = model.predict(t)
print(pred)


1/1 [==============================] - 0s 63ms/step
[[5.7532740e-01 1.5887609e-03 1.4473098e-04 7.2213128e-04 3.1566601e-02
  1.8664434e-03 2.2943923e-03 1.5039409e-04 1.9359351e-03 4.2361559e-04
  6.1082363e-02 1.1204365e-03 3.4819741e-03 1.1841088e-03 2.2140597e-03
  3.4112626e-04 9.0615510e-04 4.2598636e-04 2.0955428e-03 2.2569990e-02
  9.6515520e-04 3.3959467e-04 8.5333683e-02 1.0392197e-03 6.6639088e-02
  7.4030802e-04 5.1282350e-05 9.6389960e-04 1.2562430e-01 5.3365092e-04
  9.3047520e-06 8.3177103e-05 4.1031949e-03 1.8545653e-03 2.2547359e-04
  5.2010211e-05]]


In [22]:
from skimage.feature import hog
from skimage import exposure
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report

# Load and preprocess your images (X_train, X_test, y_train, y_test)

# Feature Extraction using HOG
hog_features = []

for image in X_train:
    fd, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(1, 1), block_norm='L2-Hys', visualize=True)
    fd = exposure.rescale_intensity(fd, in_range=(0, 10))
    hog_features.append(fd)

# Training the SVM
svm_classifier = svm.SVC(kernel='linear', C=1.0)
svm_classifier.fit(hog_features, y_train)

# Feature extraction for test data
hog_test_features = []

for image in X_test:
    fd, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(1, 1), block_norm='L2-Hys', visualize=True)
    fd = exposure.rescale_intensity(fd, in_range=(0, 10))
    hog_test_features.append(fd)

# Model Evaluation
y_pred = svm_classifier.predict(hog_test_features)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

# Classification report
report = classification_report(y_test, y_pred, target_names=labels)
print(report)


ValueError: Only images with two spatial dimensions are supported. If using with color/multichannel images, specify `channel_axis`.